## Process NEI and CCS emissions for a specific facility

This script applies the CCS emissions (without NH3 and VOC) only for the two facilities (see below). 

Author : Yunha Lee

Date: March 11, 2025

In [1]:
import geopandas as gpd
from pyproj import CRS
import os, sys

# Add the path to the main package directory
package_path = os.path.abspath('/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/LOCAETA_AQ')
if package_path not in sys.path:
    sys.path.append(package_path)

import emission_processing

# Read the point source emissions

original_emis = '/Users/yunhalee/Documents/LOCAETA/CS_emissions/Colorado_CCS_combined_NEI_point_oilgas_ptegu_ptnonimps_wo_NH3_VOC.shp'
gdf_ccs = gpd.read_file(original_emis)

# Reset index to ensure proper comparison
gdf_ccs.reset_index(drop=True, inplace=True)

target_crs = "+proj=lcc +lat_1=33.000000 +lat_2=45.000000 +lat_0=40.000000 +lon_0=-97.000000 +x_0=0 +y_0=0 +a=6370997.000000 +b=6370997.000000 +to_meter=1"
gdf_ccs = emission_processing.reproject_and_save_gdf(gdf_ccs, target_crs)


Warning 3: Cannot find header.dxf (GDAL_DATA is not defined)


Reprojecting from PROJCS["unknown",GEOGCS["unknown",DATUM["D_Unknown_based_on_Normal_Sphere_r_6370997_ellipsoid",SPHEROID["Normal_Sphere_r_6370997",6370997,0]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",40],PARAMETER["central_meridian",-97],PARAMETER["standard_parallel_1",33],PARAMETER["standard_parallel_2",45],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] to +proj=lcc +lat_1=33.000000 +lat_2=45.000000 +lat_0=40.000000 +lon_0=-97.000000 +x_0=0 +y_0=0 +a=6370997.000000 +b=6370997.000000 +to_meter=1 +type=crs


In [2]:
# Apply CCS emissions to a facility of interest
facility_eis_id = {'Suncor':1099511, 'Cherokee':3555811 } 
                   #'Suncor':1099511, } # 17445711} # Suncor frs_id = '1007923'
		# Cherokee plant alone (facility ID 1007207)  EIS_ID = 17445711
        # NEI emission without landfill facility ID 1007709


species_list = ['NOx', 'SOx', 'PM2_5'] # VOC and NH3 are excluded because the input emissions has NEI emissions for VOC and NH3. 

for key, id in facility_eis_id.items():

    gdf_emis = gdf_ccs.copy()

    if isinstance(id, int):
        print(f"{id} is integer")
    else:
        print(f"{id} must be integer")
        id = int(id)

    print(f"matching facility: {gdf_emis[gdf_emis['EIS_ID'] == id]}" )

    for spec in species_list:

        # Create a mask for where EIS_ID values are not matching
        valid_ccs_mask = gdf_emis['EIS_ID'] != id

        # Where the mask is True, replace with NEI emissions value
        gdf_emis.loc[valid_ccs_mask, spec] = gdf_emis.loc[valid_ccs_mask, spec + '_old']


    # save the final gdf     
    final_emis_file = f'/Users/yunhalee/Documents/LOCAETA/CS_emissions/Colorado_point_{key}_{id}_CCS_combined_NEI_point_oilgas_ptegu_ptnonimps_wo_NH3_VOC.shp' #Colorado_CCS_combined_NEI_point_oilgas_ptegu_ptnonimps.shp'

    if not isinstance(gdf_emis, gpd.GeoDataFrame):
        raise TypeError("The object is not a GeoDataFrame")
    else:
        if gdf_emis.crs != CRS.from_string(target_crs):
            raise ValueError(f"The GeoDataFrame CRS does not match the target CRS: {target_crs}")
        else: 
            gdf_emis.to_file(final_emis_file)
            print(f"New emission shapefile saved to: {final_emis_file}")


1099511 is integer
matching facility:          FIPS       SCC      VOC   NOx_old  NH3  SOx_old  PM2_5_old   height  \
119718  08001  10200701   8.6938  209.7100  0.0   5.5252    12.0281  22.8600   
119719  08001  30103202   0.0500    0.9400  0.0  52.9000     0.0700  45.7200   
119720  08001  30107103   5.2000    0.0000  0.0   0.0000     0.0000   0.0000   
119721  08001  30600104   2.7600   42.0200  0.0   2.8200     3.9700  18.2880   
119722  08001  30600106  24.2535  250.6707  0.0  19.3947    26.0017  13.1064   
...       ...       ...      ...       ...  ...      ...        ...      ...   
119783  08001  40700809   0.4058    0.0000  0.0   0.0000     0.0000   3.0480   
119784  08001  40704801   0.9300    0.0000  0.0   0.0000     0.0000   0.0000   
119785  08001  50410310   0.1114    5.8100  0.0   0.0000     0.0000   3.0480   
119786  08001  50410314   0.0754    2.2600  0.0   0.0100     0.0900   3.0480   
119787  08001  50410405   0.2900    0.0000  0.0   0.0000     0.0000  27.4320   

 

## Exclude a specific facility from NEI emissions

Note that Landfill is not existed in CCS emissions. 

In [3]:
import geopandas as gpd
from pyproj import CRS
import os, sys

# Add the path to the main package directory
package_path = os.path.abspath('/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/LOCAETA_AQ')
if package_path not in sys.path:
    sys.path.append(package_path)

import emission_processing

# Read the point source emissions

original_emis ='/Users/yunhalee/Documents/LOCAETA/RCM/INMAP/evaldata_v1.6.1/2020_nei_emissions/new_NEI2020_pt_oilgas_ptegu_ptnonipm.shp'
gdf_nei = gpd.read_file(original_emis)

# Reset index to ensure proper comparison
gdf_nei.reset_index(drop=True, inplace=True)

target_crs = "+proj=lcc +lat_1=33.000000 +lat_2=45.000000 +lat_0=40.000000 +lon_0=-97.000000 +x_0=0 +y_0=0 +a=6370997.000000 +b=6370997.000000 +to_meter=1"
gdf_nei = emission_processing.reproject_and_save_gdf(gdf_nei, target_crs)
print(f"Rows before filtering: {len(gdf_nei)}")

Reprojecting from PROJCS["unknown",GEOGCS["unknown",DATUM["D_Unknown_based_on_Normal_Sphere_r_6370997_ellipsoid",SPHEROID["Normal_Sphere_r_6370997",6370997,0]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",40],PARAMETER["central_meridian",-97],PARAMETER["standard_parallel_1",33],PARAMETER["standard_parallel_2",45],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] to +proj=lcc +lat_1=33.000000 +lat_2=45.000000 +lat_0=40.000000 +lon_0=-97.000000 +x_0=0 +y_0=0 +a=6370997.000000 +b=6370997.000000 +to_meter=1 +type=crs
Rows before filtering: 300008


In [4]:
# exclude the landfill facility ID 1007709 (EIS_ID = 2001411 )
facility_eis_id = {'Landfill':2001411}

for key, id in facility_eis_id.items():

    if not isinstance(id, int):
        print(f"{id} must be integer")
        id = int(id)

    print(f"Rows before filtering: {len(gdf_nei)}")

    gdf_drop = gdf_nei[gdf_nei['EIS_ID'] == id]

    print(gdf_drop)

    # Drop rows where 'EIS_ID' is equal to id
    gdf_nei = gdf_nei[gdf_nei['EIS_ID'] != id]
    
    print(f"Rows after filtering: {len(gdf_nei)}")

    # save the final gdf     
    final_emis_file = f'/Users/yunhalee/Documents/LOCAETA/RCM/INMAP/evaldata_v1.6.1/2020_nei_emissions/new_NEI2020_pt_oilgas_ptegu_ptnonipm_excluding_{key}_{id}.shp'

    if not isinstance(gdf_nei, gpd.GeoDataFrame):
        raise TypeError("The object is not a GeoDataFrame")
    else:
        if gdf_nei.crs != CRS.from_string(target_crs):
            raise ValueError(f"The GeoDataFrame CRS does not match the target CRS: {target_crs}")
        else: 
            gdf_nei.to_file(final_emis_file)
            print(f"New emission shapefile saved to: {final_emis_file}")

Rows before filtering: 300008
         FIPS       SCC     VOC    NOx  NH3   SOx    PM2_5  height     diam  \
127285  08005  20300802   11.08  71.33  0.0  2.72    9.360  8.8392  0.38710   
127286  08005  50100402   29.20   0.00  0.0  0.00  441.523  0.0000  0.00000   
127287  08005  50100410  151.25  29.61  0.0  6.57    7.310  7.6200  3.01752   

              temp  velocity   EIS_ID                       geometry  
127285  366.111110  25.63368  2001411  POINT (-653492.668 -9551.493)  
127286  255.372222   0.00000  2001411  POINT (-653492.668 -9551.493)  
127287  648.888890   3.47472  2001411  POINT (-653492.668 -9551.493)  
Rows after filtering: 300005
New emission shapefile saved to: /Users/yunhalee/Documents/LOCAETA/RCM/INMAP/evaldata_v1.6.1/2020_nei_emissions/new_NEI2020_pt_oilgas_ptegu_ptnonipm_excluding_Landfill_2001411.shp
